# Evaluation for the Real Sliding Block Experiments
This notebook can be used to load and analyze the results generated by running `training_realSlidingBlock.py`. The path to the the folder containing `ckpt.pth` and `./hydra/` needs to be specified under `path_experiment`.

In [1]:
%cd ..

/home/jan/Documents/TUDelftMSc/CVDL/project/project/PhysParamInference


In [2]:
import os
import yaml
import torch
from torchvision import utils
from models.sceneRepresentation import Scene
from dataset.dataset import ImageDataset_realData
from util.util import compute_psnr
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [3]:
# Set the path to the folder containing the results (/.hydra/, ckpt.pth)
path_experiment = os.path.join(
    os.path.abspath(''),
    'experiments',
    '2024-05-29',
    'real_world',
    'slidingBlock',
    '18-39-05'
)

In [4]:
# Load the config and the checkpoint
path_conf = os.path.join(path_experiment, '.hydra','config.yaml')
with open(path_conf) as f:
    cfg = yaml.safe_load(f)

model = Scene(**cfg['scene']['background'])

model.add_slidingBlock(
    alpha=torch.tensor(0.),
    p0=torch.ones(2),
    **cfg['scene']['local_representation']
)

path_ckpt = os.path.join(path_experiment, 'ckpt.pth')
model.load_state_dict(torch.load(path_ckpt))

model.use_homography = cfg['homography']['enable']

model.to(device)
print("Moved to device")

Moved to device


In [5]:
# Load the data
data_path = os.path.join(os.path.abspath(''), 'data',cfg['data']['path_data'])

train_data = ImageDataset_realData(
    **cfg['data'],
    max_samples=cfg['data']['samples_train']
)

dataset = ImageDataset_realData(
    **cfg['data'],
    indices=train_data.unused_inds
)

tspan = dataset.t_steps.to(device)
print(f"Tspan eval data: {tspan}")
print(f"Tspan train data: {train_data.t_steps}")

Tspan eval data: tensor([0.0000, 0.0333, 0.1000, 0.1667, 0.2333, 0.3000, 0.3667, 0.4000],
       device='cuda:0')
Tspan train data: tensor([0.0000, 0.0667, 0.1333, 0.2000, 0.2667, 0.3333])


/home/jan/anaconda3/envs/physParamInference/lib/python3.8/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
# Render images and evaluate
model.eval()
model.update_trafo(tspan)
H, W = dataset.get_image_dim()
output = model.render_image(W, H)
ims = output["Image"].cpu()
masks = output['Mask'].cpu()

psnr = compute_psnr(ims, dataset.get_full_images())
norm = torch.norm(torch.eye(3) - model.homography_matrix.cpu())

measured_alpha = 24
rel_error_alpha = torch.abs(torch.rad2deg(model.local_representation.ode.alpha) - measured_alpha) / measured_alpha
print(f"PSNR: {psnr}, Norm diff: {norm}")
print(f"Rel Error alpha: {100*rel_error_alpha:.2f}%")
print(f"{psnr:.2f} & {norm:.2f}")

PSNR: 27.770885467529297, Norm diff: 0.00722624221816659
Rel Error alpha: 90.89%
27.77 & 0.01


In [7]:
# Save the renderings to the experiments folder
path_folder_rendering = os.path.join(path_experiment, 'renderingsRealWorld')
os.makedirs(path_folder_rendering)

# Save individual images
for i in range(len(tspan)):
    path = os.path.join(path_folder_rendering, f"{i+1}_eval.jpg")
    cur_im = ims[i].permute(2, 0, 1)
    utils.save_image(cur_im, path)

    path = os.path.join(path_folder_rendering, f"{i+1}_mask_eval.jpg")
    cur_mask = masks[i]
    utils.save_image(cur_mask, path)

    path = os.path.join(path_folder_rendering, f"{i+1}_gt.jpg")
    cur_im_gt = dataset.get_full_images(i).permute(2, 0, 1)
    utils.save_image(cur_im_gt, path)